In [1]:
import networkx as nx
import os
import geopandas as gpd
import pandas as pd
from pathlib import Path
from shapely import LineString
import numpy as np
import requests
%matplotlib inline

BASE_DIR = (Path.cwd()).parent.parent

In [2]:
naRailNodesDf = gpd.read_file( BASE_DIR / "inputs/NTAD_North_American_Rail_Network_Nodes")
naRailLinesDf = gpd.read_file( BASE_DIR / "inputs/NTAD_North_American_Rail_Network_Lines")
fafZonesDf = gpd.read_file( BASE_DIR / "inputs/2017_CFS_Metro_Areas_with_FAF").to_crs(3857)

# Create Graph

In [ ]:
naRailNodesWithFafZonesDf = naRailNodesDf.sjoin(fafZonesDf, how='left', predicate="within")

# .to_crs(3857)
fafZoneNodesDf = naRailNodesWithFafZonesDf.dissolve(['FAF_Zone']).reset_index()
fafZoneNodesDf.geometry = fafZoneNodesDf.centroid
fafZoneNodesDf.head()

In [ ]:
naRailLinesWithFafZonesDf = naRailLinesDf.join(naRailNodesWithFafZonesDf.set_index('FRANODEID').add_suffix('_fr'), on='FRFRANODE')
naRailLinesWithFafZonesDf = naRailLinesWithFafZonesDf.join(naRailNodesWithFafZonesDf.set_index('FRANODEID').add_suffix('_to'), on='TOFRANODE')

naRailLinesWithFafZonesDf = naRailLinesWithFafZonesDf[naRailLinesWithFafZonesDf.columns[
  ~(
    naRailLinesWithFafZonesDf.columns.str.endswith('_fr')
    | naRailLinesWithFafZonesDf.columns.str.endswith('_to')
  )
].union(['FAF_Zone_fr', 'FAF_Zone_to'])]

naRailLinesWithFafZonesDf.head()

In [ ]:
fafZoneLinksDf = naRailLinesWithFafZonesDf[naRailLinesWithFafZonesDf.FAF_Zone_fr != naRailLinesWithFafZonesDf.FAF_Zone_to].dropna(subset=['FAF_Zone_fr', 'FAF_Zone_to'])
fafZoneLinksDf = fafZoneLinksDf.dissolve(by=['FAF_Zone_fr','FAF_Zone_to'])
fafZoneLinksDf = fafZoneLinksDf.join(fafZoneNodesDf.set_index('FAF_Zone').add_suffix('_fr'), on='FAF_Zone_fr')
fafZoneLinksDf = fafZoneLinksDf.join(fafZoneNodesDf.set_index('FAF_Zone').add_suffix('_to'), on='FAF_Zone_to')
fafZoneLinksDf.geometry = fafZoneLinksDf.apply(lambda r:  LineString([
    [r.geometry_fr.x, r.geometry_fr.y],
    [r.geometry_to.x, r.geometry_to.y]
]), axis=1)

In [6]:
railnet = nx.Graph()

In [84]:
aggNetworkNodesDf = pd.read_excel( BASE_DIR / 'inputs'/'AggregateNetwork.xlsx', sheet_name='Nodes').set_index('Node')
aggNetworkNodesDf = gpd.GeoDataFrame(
  aggNetworkNodesDf,
  geometry=gpd.points_from_xy(aggNetworkNodesDf.Longitude, aggNetworkNodesDf.Latitude),
  crs="EPSG:4326"
)


aggNetworkLinksDf = pd.read_excel( BASE_DIR / 'inputs'/'AggregateNetwork.xlsx', sheet_name='Links')
aggNetworkLinksDf = aggNetworkLinksDf.set_index('Start_Node').join(aggNetworkNodesDf.add_suffix('_fr'))
aggNetworkLinksDf = aggNetworkLinksDf.set_index('End_Node').join(aggNetworkNodesDf.add_suffix('_to'))
aggNetworkLinksDf = gpd.GeoDataFrame(
  aggNetworkLinksDf,
  geometry=aggNetworkLinksDf.apply(lambda r: LineString([
    [r.geometry_fr.x,r.geometry_fr.y],
    [r.geometry_to.x,r.geometry_to.y]
  ]), axis=1),
  crs="EPSG:4326"
)

In [ ]:
node_list = []
# with arcpy.da.SearchCursor(faf_layer_name, ['FAF_Zone', 'FAF_Zone_1', 'INTPTLAT', 'INTPTLON'] ) as cursor:
#     for faf_id, name, lat, lon in cursor:
#         node_list.append((int(faf_id), {'faf_id': int(faf_id), 'name': name, 'lat': float(lat), 'lon': float(lon)}))
# railnet.add_nodes_from(node_list)
# for r in naRailNodesWithFafZonesDf:
#   print(r)
# railnet.add_nodes_from

In [1]:
import math

# node_fields = [f.name for f in arcpy.ListFields(line_layer_name)]
links = {}

def add_to_links_dict(link):
    fr = int(link.FAF_Zone_fr)
    to = int(link.FAF_Zone_to)
    if math.isnan(fr) or math.isnan(to): return
    pair = (int(fr), int(to))
    if pair not in links:
        links[pair] = 0
    links[pair] += 1
    return link

# naRailLinesWithFafZonesDf
# railnet.add_weighted_edges_from([(fr, to, links[(fr, to)]) for fr,to in links])

In [65]:
nx.write_gml(railnet,'Data/faf_railnet.gml')

# Load onto map

In [3]:
railnet = nx.read_gml('Data/faf_railnet.gml')

In [5]:
sr = arcpy.Describe(faf_layer_name).spatialReference
arcpy.env.outputCoordinateSystem = sr

In [ ]:
sr

In [ ]:
faf_nodes_fc = f'{arcpy.env.workspace}/faf_nodes'
faf_edges_fc = f'{arcpy.env.workspace}/faf_edges'
arcpy.management.Delete('faf_nodes')
arcpy.management.Delete('faf_edges')

In [ ]:
arcpy.conversion.ExportFeatures('Data/faf_network_nodes.shp', faf_nodes_fc)
arcpy.conversion.ExportFeatures('Data/faf_network_edges.shp', faf_edges_fc)